# Deep Seeeion 8주차 과제

tweets를 기반으로 sentiment를 예측하는 task를 수행하고자 합니다. 

1. 데이터셋에 맞는 전처리를 진행해주세요.
2. 실습 파일을 참고하여 Sentiment_RNN, Sentiment_LSTM, Sentiment_GRU class를 각각 완성해주세요.
3. 3가지 모델에서의 성능을 비교해주세요.

In [12]:
inputs = {a.cuda(), b.cuda(), c.cuda()}
outputs = model(**inputs)
del inputs
torch.cuda.empty_cache()

NameError: name 'a' is not defined

In [11]:
import GPUtil 
GPUtil.showUtilization()


| ID | GPU | MEM |
------------------
|  0 |  0% |  3% |


In [10]:
import torch

In [2]:
torch.cuda.memory_summary(device=None, abbreviated=False)

'|===========================================================================|\n|                  PyTorch CUDA memory summary, device ID 0                 |\n|---------------------------------------------------------------------------|\n|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |\n|===========================================================================|\n|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |\n|---------------------------------------------------------------------------|\n| Allocated memory      |       0 B  |       0 B  |       0 B  |       0 B  |\n|       from large pool |       0 B  |       0 B  |       0 B  |       0 B  |\n|       from small pool |       0 B  |       0 B  |       0 B  |       0 B  |\n|---------------------------------------------------------------------------|\n| Active memory         |       0 B  |       0 B  |       0 B  |       0 B  |\n|       from large pool |       0 B  |       0 B  |

In [1]:
import pandas as pd
import numpy as np
import random
import re

import torch
import torch.nn as nn

from tensorflow.keras.preprocessing.sequence import pad_sequences

from sklearn.model_selection import train_test_split

### Reading Data

In [4]:
tweets = pd.read_csv('Tweets.csv')
tweets.head()

,textID,text,selected_text,sentiment
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative
2,088c60f138,my boss is bullying me...,bullying me,negative
3,9642c003ef,what interview! leave me alone,leave me alone,negative
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative


In [5]:
#na값 제거 해줘야함
tweets.dropna(how='any',axis=0,inplace=True)

In [15]:
tweets.sentiment.unique()

array(['neutral', 'negative', 'positive'], dtype=object)

In [16]:
# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(tweets['text'], tweets['sentiment'], stratify=tweets['sentiment'], random_state=220510, test_size=.2)

# twitter roberta tokenizer

In [11]:
from transformers import AutoTokenizer, RobertaForSequenceClassification, Trainer, TrainingArguments, DataCollatorWithPadding
import pandas as pd
import numpy as np
from datasets import load_dataset, load_metric

#MODEL = "klue/roberta-base"
INPUT = "Tweets.csv"
MAX_LEN = 256
dataset = load_dataset("csv", data_files=INPUT,split='train')
tokenizer = AutoTokenizer.from_pretrained("cardiffnlp/twitter-roberta-base-sentiment")
def example_fn(examples):
    outputs = tokenizer(examples['text'],padding=True, max_length=MAX_LEN,truncation=True)
    if 'type' in examples:
        outputs["type"] = examples["sentiment"]
    return outputs

dataset = dataset.map(example_fn, remove_columns=['text', 'type'])

dataset = dataset.train_test_split(0.2)


Using custom data configuration default-36f36e6f271ef338
Reusing dataset csv (C:\Users\82103\.cache\huggingface\datasets\csv\default-36f36e6f271ef338\0.0.0\433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519)


ValueError: Column to remove ['type'] not in the dataset. Current columns in the dataset: ['Unnamed: 0', 'textID', 'text', 'selected_text', 'sentiment']

In [10]:
dataset

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'textID', 'selected_text', 'input_ids', 'attention_mask'],
        num_rows: 21984
    })
    test: Dataset({
        features: ['Unnamed: 0', 'textID', 'selected_text', 'input_ids', 'attention_mask'],
        num_rows: 5496
    })
})

In [9]:
X_train=pd.DataFrame(dataset['train'])['input_ids']
X_test=pd.DataFrame(dataset['test'])['input_ids']
y_train=pd.DataFrame(dataset['train'])['sentiment']
y_test=pd.DataFrame(dataset['test'])['sentiment']

KeyError: 'sentiment'

### Preprocessing

In [17]:
def preprocessing(data):
    data = data.map(lambda x: x.lower())
    return data

### Tokenization

In [18]:
# 띄어쓰기 기준 tokenization
def sep_based_tok(sentences):
    toks = sentences.map(lambda x: x.split())
    return toks

### Word Embedding

In [19]:
# label encoding
def le_emb(toks, token_cb):
    le = {token:i for i, token in enumerate(token_cb)}
    embs = toks.map(lambda x: [le.get(tok, 0) for tok in x])
    return embs

In [20]:
def sentence_vectorization(sentences, tok_method, emb_method):
    sentences = preprocessing(sentences) # preprocessing
    toks = tok_method(sentences) # tokenization
    token_cb = ['<unk>'] + list({word for sentence in toks for word in sentence}) # make vocabulary
    embs = emb_method(toks, token_cb) # word embedding
    return embs

In [21]:
X_train = sentence_vectorization(X_train, sep_based_tok, le_emb)
X_test = sentence_vectorization(X_test, sep_based_tok, le_emb)

### Modeling

In [3]:
# 장비 할당
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

In [33]:
# 패딩: 입력 벡터 길이 맞추기
X_train = torch.FloatTensor(pad_sequences(X_train)).unsqueeze(2)
X_test = torch.FloatTensor(pad_sequences(X_test)).unsqueeze(2)

In [34]:
# 타겟 데이터 전처리
idx2label = dict(enumerate(y_train.unique()))
label2idx = {label:idx for idx, label in idx2label.items()}
y_train = [label2idx[x] for x in y_train]
y_test = [label2idx[x] for x in y_test]
y_train = torch.LongTensor(y_train)
y_test = torch.LongTensor(y_test)

In [35]:
from torch.utils.data import TensorDataset

# Combine the training inputs into a TensorDataset.
train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)

train_dataset, val_dataset = train_test_split(train_dataset, random_state=220510, test_size=.2)

In [37]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

# The DataLoader needs to know our batch size for training, so we specify it 
batch_size = 32

# Create the DataLoaders for our training and validation sets.
# We'll take training samples in random order. 
train_dataloader = DataLoader(
            train_dataset,  # The training samples.
            sampler = RandomSampler(train_dataset), # Select batches randomly
            batch_size = batch_size # Trains with this batch size.
        )

# For validation the order doesn't matter, so we'll just read them sequentially.
val_dataloader = DataLoader(
            val_dataset, # The validation samples.
            sampler = SequentialSampler(val_dataset), # Pull out batches sequentially.
            batch_size = batch_size # Evaluate with this batch size.
        )

test_dataloader = DataLoader(
            test_dataset, # The validation samples.
            sampler = SequentialSampler(val_dataset), # Pull out batches sequentially.
            batch_size = batch_size # Evaluate with this batch size.
        )

## RNN

In [42]:

class Sentiment_RNN(nn.Module):
    def __init__(self, input_size=1, hidden_size=32, num_layers=5, dropout=.3, bidirectional=True):
        super(Sentiment_RNN, self).__init__()
        self.gru = nn.RNN(input_size=input_size,
                            hidden_size = hidden_size,
                            num_layers = num_layers,
                            dropout = dropout,
                            bidirectional=bidirectional)
        if bidirectional:
            self.fc = nn.Linear(hidden_size*2, 4)
        else:
            self.fc = nn.Linear(hidden_size, 4)
    def forward(self, x):
        output, hidden = self.gru(x)
        last_output = output[:,-1,:]
        return self.fc(last_output)

## LSTM

In [43]:

class Sentiment_LSTM(nn.Module):
    def __init__(self, input_size=1, hidden_size=32, num_layers=5, dropout=.3, bidirectional=True):
        super(Sentiment_LSTM, self).__init__()
        self.gru = nn.LSTM(input_size=input_size,
                            hidden_size = hidden_size,
                            num_layers = num_layers,
                            dropout = dropout,
                            bidirectional=bidirectional)
        if bidirectional:
            self.fc = nn.Linear(hidden_size*2, 4)
        else:
            self.fc = nn.Linear(hidden_size, 4)
    def forward(self, x):
        output, hidden = self.gru(x)
        last_output = output[:,-1,:]
        return self.fc(last_output)

## GRU

In [38]:

class Sentiment_GRU(nn.Module):
    def __init__(self, input_size=1, hidden_size=32, num_layers=5, dropout=.3, bidirectional=True):
        super(Sentiment_GRU, self).__init__()
        self.gru = nn.GRU(input_size=input_size,
                            hidden_size = hidden_size,
                            num_layers = num_layers,
                            dropout = dropout,
                            bidirectional=bidirectional)
        if bidirectional:
            self.fc = nn.Linear(hidden_size*2, 4)
        else:
            self.fc = nn.Linear(hidden_size, 4)
    def forward(self, x):
        output, hidden = self.gru(x)
        last_output = output[:,-1,:]
        return self.fc(last_output)

In [39]:
def train(model, train_dataloader, optim, loss_fc):
    model.train()
    train_loss = 0
    correct = 0
    num_samples = 0
    for idx, batch in enumerate(train_dataloader):
        text = batch[0].to(device)
        target = batch[1].to(device)
        
        optimizer.zero_grad()
        preds = model(text)
        loss = loss_fc(preds, target)
        train_loss += loss.item()
        loss.backward()
        optimizer.step()
        
        prediction = preds.max(1, keepdim=True)[1]
        correct += prediction.eq(target.view_as(prediction)).sum().item()
        
        num_samples += target.size(0)
        
    train_loss /= num_samples
    train_acc = 100 * correct /num_samples
    
    print(f'Train Loss : {train_loss:.4} | Train Acc : {train_acc:.4}')

def evaluate(model, test_dataloader, loss_fc):
    model.eval() # 모델을 평가상태로 지정
    test_loss = 0
    correct = 0
    num_samples = 0
    with torch.no_grad(): # 평가 과정에서 gradient 업데이트를 하지 않기 위해
        for batch in test_dataloader:
            text = batch[0].to(device)
            target = batch[1].to(device)
            output = model(text)
            test_loss += criterion(output, target).item()
            
            prediction = output.max(1, keepdim=True)[1] # 벡터 값 내 최대값으로 예측
            correct += prediction.eq(target.view_as(prediction)).sum().item()
            
            num_samples += target.size(0)
            
    test_loss /= num_samples
    test_accuracy = 100 * correct /num_samples
    
    return test_loss, test_accuracy

In [40]:
def train_epochs(model, train_dataloader, val_dataloader, optim, loss_fc, n_epoch):
    for epoch in range(n_epoch):
        print(f'-----Epoch : {epoch+1}-----')
        train(model, train_dataloader, optim, loss_fc)
        valid_loss, valid_acc = evaluate(model, val_dataloader, loss_fc)
        print(f'Valid Loss : {valid_loss:.4} | Valid Acc : {valid_acc:.4}')

In [41]:
torch.manual_seed(0)
torch.cuda.manual_seed(0)
torch.cuda.manual_seed_all(0)
np.random.seed(0)
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True
random.seed(0)

# 모델을 변경해서 사용해주세요.
model = Sentiment_LSTM().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)
criterion = torch.nn.CrossEntropyLoss()

train_epochs(model, train_dataloader, val_dataloader, optimizer, criterion, 10)

-----Epoch : 1-----
Train Loss : 0.04181 | Train Acc : 33.27
Valid Loss : 0.03853 | Valid Acc : 39.3
-----Epoch : 2-----
Train Loss : 0.03743 | Train Acc : 40.41
Valid Loss : 0.03632 | Valid Acc : 39.55
-----Epoch : 3-----
Train Loss : 0.03587 | Train Acc : 40.48
Valid Loss : 0.03547 | Valid Acc : 39.55
-----Epoch : 4-----
Train Loss : 0.03521 | Train Acc : 40.43
Valid Loss : 0.03502 | Valid Acc : 39.55
-----Epoch : 5-----
Train Loss : 0.03486 | Train Acc : 40.36
Valid Loss : 0.03479 | Valid Acc : 39.25
-----Epoch : 6-----
Train Loss : 0.03464 | Train Acc : 40.31
Valid Loss : 0.03466 | Valid Acc : 39.25
-----Epoch : 7-----
Train Loss : 0.03449 | Train Acc : 40.43
Valid Loss : 0.03456 | Valid Acc : 39.25
-----Epoch : 8-----
Train Loss : 0.03439 | Train Acc : 40.18
Valid Loss : 0.03451 | Valid Acc : 39.53
-----Epoch : 9-----
Train Loss : 0.03431 | Train Acc : 40.42
Valid Loss : 0.03447 | Valid Acc : 39.55
-----Epoch : 10-----
Train Loss : 0.03428 | Train Acc : 40.28
Valid Loss : 0.03443 

### Prediction

In [44]:
#GTU
test_loss, test_acc = evaluate(model, test_dataloader, criterion)
print(f'Test Loss : {test_loss:.4} | Test Acc : {test_acc:.4}')

Test Loss : 0.03433 | Test Acc : 40.39
